In [1]:
!pip install sentence-transformers
!pip install faiss-cpu

In [3]:
# 이전 코드 파트 (how to make vector DB 보면 주석이 더 자세히 달려있음)
import os
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer


directory_path = r"/Users/jeong-uihyeon/chatbot/file/Sample/02.라벨링데이터"  #원하는 경로 (일단 한정해 두었음)

def load_json(file_path):
    try:
        with open(file_path, 'r') as file:
            data = json.load(file) # 전체 내용
    except FileNotFoundError:
        data = "파일을 찾을 수 없습니다."
    except json.JSONDecodeError:
        data = "유효하지 않은 JSON 형식입니다."
    return data


def read_json_files(directory):
    question = []
    answer = []
    
    # 디렉토리와 하위 디렉토리를 탐색
    for root, dirs, files in os.walk(directory):
        for file in files:
            # 파일 확장자가 .json인지 확인
            if file.endswith('.json'):
                file_path = os.path.join(root, file)
                # JSON 파일 읽기
                data = load_json(file_path)
                # json파일에서 question / answer 쌍으로 추출
                question.append(data['question']["comment"])
                answer.append(data['answer']["comment"])
                
    return question, answer


def embed_text(texts, model):
    return model.encode(texts, show_progress_bar=True)


# 질문 답변 쌍으로 저장
questions, answers = read_json_files(directory_path)

# Sentence-BERT 모델 로드
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# 결합된 텍스트를 벡터로 변환
combined_vectors = embed_text(questions, model)

# 질문 전부를 임베딩하기
qustion_vectors = model.encode(questions)

# 질문 벡터의 차원 확인
vector_dimension = len(qustion_vectors[0])

#Faiss index 생성
index = faiss.IndexFlatL2(vector_dimension)

# 인덱스에 벡터 추가
index.add(np.array(qustion_vectors).astype('float32'))

/Users/jeong-uihyeon/chatbot/.venv/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Batches: 100%|██████████| 138/138 [00:19<00:00,  6.98it/s]


In [11]:
def search_qa(query, k=1):
    # 질문(query)를 vector로 변환
    query_vector = model.encode([query])

    # distances는 query와 가장 가까운 값의 유사도
    # indices는 query와 가장 가까운 값의 인덱스 값
    distances, indices = index.search(np.array(query_vector).astype('float32'), k)
    return [(questions[i], answers[i], distances[0][j]) for j, i in enumerate(indices[0])] # k값이 복수일 경우 값들을 불러줌

results = search_qa('그러니까 내가 아까 얘기한 부산지검의 명예를 회복하기 위해서 해야 된다는 얘기이고 그 이전의 지검 검사장도 이 지역사회에서 유착되었다는 설도 여기에서는 널리 유포되어 있었습니다. 그런데다가 최근에 두 지검 검사장이 이 사건의 변호인으로 선임되고 하니까 점점 부산지검은 다대ㆍ만덕사건에 관한 한 처음부터 수사를 못 하게 되어 있는 사건이다, 내가 아까 말한 대로 국정감사 때마다 요구하고 추궁하고 또 감사원 감사가 착수되고 하니까 할 수 없이 등 떠밀려서 마지못해서 하는 것 아니냐 하는 것입니다.  수사진은 왜 교체했습니까?')

print(f"question: {results[0][0]}")
print(f"answer: {results[0][1]}")
print(f"distance: {results[0][2]}") # 쿼리 벡터와 찾은 질문 벡터 사이의 거리 (유사도)

question: 그러니까 내가 아까 얘기한 부산지검의 명예를 회복하기 위해서 해야 된다는 얘기이고 그 이전의 지검 검사장도 이 지역사회에서 유착되었다는 설도 여기에서는 널리 유포되어 있었습니다. 그런데다가 최근에 두 지검 검사장이 이 사건의 변호인으로 선임되고 하니까 점점 부산지검은 다대ㆍ만덕사건에 관한 한 처음부터 수사를 못 하게 되어 있는 사건이다, 내가 아까 말한 대로 국정감사 때마다 요구하고 추궁하고 또 감사원 감사가 착수되고 하니까 할 수 없이 등 떠밀려서 마지못해서 하는 것 아니냐 하는 것입니다.  수사진은 왜 교체했습니까?
answer: 인사가 된 것이 금년 2월인데 그때는 이영복이가 이미 도주했고 나머지 사람들에 대해서는 1월 중에 다 기소된 뒤였습니다. 그렇기 때문에 정기인사에 따라서 부장검사가 서울로 갈 차례이고 또 수사검사는 부부장으로 승진을 하게 되어 있었습니다.  저희들 관행에 따르면 부부장검사로 승진을 하면 다른 청에 가게 되어 있습니다. 그렇게 해서 가기는 했습니다마는 후임 부장이나 후임 검사들의 수사열의는 또 수사에 대한 파악도는 전임자들에 결코 못지않다는 것을 말씀드립니다.
distance: 1.8244607213890873e-11
